In [315]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler 
import seaborn as sns 

In [316]:
train = pd.read_csv(r"C:/Users\dai\Desktop\Machine Learning\Datasets\train_v9rqX0R.csv")
test = pd.read_csv(r"C:/Users\dai\Desktop\Machine Learning\Datasets\test_AbJTz2l.csv")

In [317]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [318]:
train['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [319]:
train['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: count, dtype: int64

In [320]:
train['Item_Fat_Content'].replace({'reg': 'Regular', 'LF': 'Low Fat','low fat' :'Low Fat'},inplace=True)
#test['Item_Fat_Content'] = test['Item_Fat_Content'].replace({'reg': 'Regular', 'LF': 'Low Fat','low fat' :'Low Fat'},inplace=True)

In [321]:
train['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    5517
Regular    3006
Name: count, dtype: int64

In [322]:
print(train['Item_Weight'].isnull().sum())
print(train['Item_Weight'].notnull().sum())


1463
7060


In [323]:
train.sort_values(by='Item_Identifier',ascending=True)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
8043,DRA12,11.60,Low Fat,0.068535,Soft Drinks,143.0154,OUT010,1998,NaN,Tier 3,Grocery Store,283.6308
118,DRA12,11.60,Low Fat,0.041178,Soft Drinks,140.3154,OUT017,2007,NaN,Tier 2,Supermarket Type1,2552.6772
1693,DRA12,11.60,Low Fat,0.000000,Soft Drinks,141.9154,OUT035,2004,Small,Tier 2,Supermarket Type1,992.7078
1197,DRA12,11.60,Low Fat,0.000000,Soft Drinks,141.6154,OUT045,2002,NaN,Tier 2,Supermarket Type1,3829.0158
1245,DRA12,11.60,Low Fat,0.040912,Soft Drinks,142.3154,OUT013,1987,High,Tier 3,Supermarket Type1,2552.6772
...,...,...,...,...,...,...,...,...,...,...,...,...
7926,NCZ54,14.65,Low Fat,0.083489,Household,161.3552,OUT049,1999,Medium,Tier 1,Supermarket Type1,7148.0288
214,NCZ54,14.65,Low Fat,0.000000,Household,161.5552,OUT010,1998,NaN,Tier 3,Grocery Store,324.9104
1896,NCZ54,NaN,Low Fat,0.082956,Household,164.0552,OUT027,1985,Medium,Tier 3,Supermarket Type3,5523.4768
7166,NCZ54,NaN,Low Fat,0.145952,Household,160.9552,OUT019,1985,Small,Tier 1,Grocery Store,162.4552


In [324]:
items_trn = train[['Item_Identifier','Item_Weight']].sort_values(by='Item_Identifier')
items_trn = items_trn[items_trn['Item_Weight'].notna()]

In [325]:
items_tst = test[['Item_Identifier','Item_Weight']].sort_values(by='Item_Identifier')
items_tst = items_tst[items_tst['Item_Weight'].notna()]

In [326]:
items = pd.concat([items_trn,items_tst])
items = items.drop_duplicates()
train.drop('Item_Weight',axis=1,inplace=True)

In [327]:
items.isnull().sum()

Item_Identifier    0
Item_Weight        0
dtype: int64

In [328]:
train_wt = train.merge(items,how='inner',on='Item_Identifier')

In [329]:
outlets_trn = train[['Outlet_Identifier','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type']]
outlets_tst = test[['Outlet_Identifier','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type']]

In [330]:
outlets = pd.concat([outlets_trn,outlets_tst])
outlets.drop_duplicates(inplace=True)
outlets.head()

,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,OUT018,2009,Medium,Tier 3,Supermarket Type2
3,OUT010,1998,NaN,Tier 3,Grocery Store
4,OUT013,1987,High,Tier 3,Supermarket Type1
7,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [331]:
train_wt['Outlet_Size'].fillna(value='Small',inplace=True)
train_wt.isnull().sum()

Item_Identifier              0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
Item_Weight                  0
dtype: int64

In [332]:
train.head()

,Item_Identifier,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [333]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Columns: 1593 entries, Item_Visibility to Outlet_Type_Supermarket Type3
dtypes: bool(1590), float64(2), int64(1)
memory usage: 13.1 MB


In [334]:
train.isnull().sum()

Item_Identifier                 0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [335]:
X = train.drop('Item_Outlet_Sales', axis=1)
X=pd.get_dummies(X,drop_first=True)
y = train['Item_Outlet_Sales']
X.head()

,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Identifier_DRA24,Item_Identifier_DRA59,Item_Identifier_DRB01,Item_Identifier_DRB13,Item_Identifier_DRB24,Item_Identifier_DRB25,Item_Identifier_DRB48,...,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,0.016047,249.8092,1999,False,False,False,False,False,False,False,...,False,False,True,True,False,False,False,True,False,False
1,0.019278,48.2692,2009,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
2,0.016760,141.6180,1999,False,False,False,False,False,False,False,...,False,False,True,True,False,False,False,True,False,False
3,0.000000,182.0950,1998,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,0.000000,53.8614,1987,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False


In [336]:
std = StandardScaler().set_output(transform='pandas')
std.fit_transform(X)

,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Identifier_DRA24,Item_Identifier_DRA59,Item_Identifier_DRB01,Item_Identifier_DRB13,Item_Identifier_DRB24,Item_Identifier_DRB25,Item_Identifier_DRB48,...,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,-0.970732,1.747454,0.139541,-0.02867,-0.030652,-0.018765,-0.024228,-0.021669,-0.026542,-0.02867,...,-0.349762,-0.349973,2.857362,1.432326,-0.623893,-0.696678,-0.804732,0.726802,-0.349551,-0.351028
1,-0.908111,-1.489023,1.334103,-0.02867,-0.030652,-0.018765,-0.024228,-0.021669,-0.026542,-0.02867,...,-0.349762,-0.349973,-0.349973,1.432326,-0.623893,-0.696678,1.242650,-1.375891,2.860816,-0.351028
2,-0.956917,0.010040,0.139541,-0.02867,-0.030652,-0.018765,-0.024228,-0.021669,-0.026542,-0.02867,...,-0.349762,-0.349973,2.857362,1.432326,-0.623893,-0.696678,-0.804732,0.726802,-0.349551,-0.351028
3,-1.281758,0.660050,0.020085,-0.02867,-0.030652,-0.018765,-0.024228,-0.021669,-0.026542,-0.02867,...,-0.349762,-0.349973,-0.349973,-0.698165,-0.623893,-0.696678,1.242650,-1.375891,-0.349551,-0.351028
4,-1.281758,-1.399220,-1.293934,-0.02867,-0.030652,-0.018765,-0.024228,-0.021669,-0.026542,-0.02867,...,-0.349762,-0.349973,-0.349973,-0.698165,-0.623893,-0.696678,1.242650,0.726802,-0.349551,-0.351028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,-0.181193,1.180783,-1.293934,-0.02867,-0.030652,-0.018765,-0.024228,-0.021669,-0.026542,-0.02867,...,-0.349762,-0.349973,-0.349973,-0.698165,-0.623893,-0.696678,1.242650,0.726802,-0.349551,-0.351028
8519,-0.371154,-0.527301,0.497909,-0.02867,-0.030652,-0.018765,-0.024228,-0.021669,-0.026542,-0.02867,...,2.859087,-0.349973,-0.349973,-0.698165,-0.623893,1.435383,-0.804732,0.726802,-0.349551,-0.351028
8520,-0.599784,-0.897208,0.736822,-0.02867,-0.030652,-0.018765,-0.024228,-0.021669,-0.026542,-0.02867,...,-0.349762,-0.349973,-0.349973,-0.698165,1.602840,1.435383,-0.804732,0.726802,-0.349551,-0.351028
8521,1.532880,-0.607977,1.334103,-0.02867,-0.030652,-0.018765,-0.024228,-0.021669,-0.026542,-0.02867,...,-0.349762,-0.349973,-0.349973,1.432326,-0.623893,-0.696678,1.242650,-1.375891,2.860816,-0.351028


In [337]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

cat = list(X.select_dtypes(include=object).columns)
c_gbr = CatBoostRegressor(random_state=24, cat_features=cat)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                   test_size = 0.3, 
                                   random_state=24)

In [338]:
c_gbr.fit(X_train, y_train)
y_pred = c_gbr.predict(X_test)

Learning rate set to 0.054292
0:	learn: 1662.4907911	total: 16.4ms	remaining: 16.4s
1:	learn: 1615.6053277	total: 29.6ms	remaining: 14.8s
2:	learn: 1571.0742795	total: 40.7ms	remaining: 13.5s
3:	learn: 1530.1204287	total: 70.3ms	remaining: 17.5s
4:	learn: 1490.9758274	total: 97.7ms	remaining: 19.4s
5:	learn: 1457.1261493	total: 114ms	remaining: 18.8s
6:	learn: 1425.2956699	total: 132ms	remaining: 18.8s
7:	learn: 1397.9934195	total: 145ms	remaining: 17.9s
8:	learn: 1371.2730939	total: 156ms	remaining: 17.2s
9:	learn: 1346.2063490	total: 167ms	remaining: 16.5s
10:	learn: 1325.9020930	total: 182ms	remaining: 16.3s
11:	learn: 1304.0968905	total: 196ms	remaining: 16.1s
12:	learn: 1283.0411781	total: 206ms	remaining: 15.6s
13:	learn: 1263.9827907	total: 216ms	remaining: 15.2s
14:	learn: 1249.1238957	total: 225ms	remaining: 14.8s
15:	learn: 1235.8269503	total: 236ms	remaining: 14.5s
16:	learn: 1221.7513666	total: 247ms	remaining: 14.3s
17:	learn: 1208.3363551	total: 258ms	remaining: 14.1s
18:

In [339]:
from sklearn.metrics import r2_score


y_pred = c_gbr.predict(X_test)
print(("r2 score(svm):"), r2_score(y_test, y_pred))

r2 score(svm): 0.5809836501243664


In [ ]:
from sklearn.model_selection import KFold , GridSearchCV


params = {'learning_rate': np.linspace(0.001, 0.9, 10), 'max_depth': [None, 2, 3, 4], 'n_estimators': [25,50, 100]}
kfold = KFold(n_splits=5, shuffle=True, random_state=24)
c_gbr = CatBoostRegressor (random_state=24, cat_features=cat)
gcv = GridSearchCV(c_gbr, param_grid=params, cv=kfold, n_jobs=-1)
gcv.fit(X, y)

print(gcv.best_params_)
print(gcv.best_score_)

0:	learn: 1519.9944389	total: 11.2ms	remaining: 269ms
1:	learn: 1392.2606731	total: 22.6ms	remaining: 260ms
2:	learn: 1291.4406921	total: 33.4ms	remaining: 245ms
3:	learn: 1232.8351084	total: 44.1ms	remaining: 232ms
4:	learn: 1182.3402071	total: 55.6ms	remaining: 222ms
5:	learn: 1149.6016409	total: 66.3ms	remaining: 210ms
6:	learn: 1124.3426932	total: 76.9ms	remaining: 198ms
7:	learn: 1107.4861697	total: 87.6ms	remaining: 186ms
8:	learn: 1096.3408724	total: 98.3ms	remaining: 175ms
9:	learn: 1089.4056936	total: 109ms	remaining: 163ms
10:	learn: 1084.1729996	total: 120ms	remaining: 153ms
11:	learn: 1079.0448307	total: 131ms	remaining: 142ms
12:	learn: 1075.7783550	total: 142ms	remaining: 131ms
13:	learn: 1074.3818848	total: 153ms	remaining: 120ms
14:	learn: 1073.5387320	total: 164ms	remaining: 110ms
15:	learn: 1072.4916766	total: 176ms	remaining: 98.8ms
16:	learn: 1071.8092878	total: 187ms	remaining: 87.8ms
17:	learn: 1069.7671854	total: 199ms	remaining: 77.5ms
18:	learn: 1067.7488872	to

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(learning_rate = 0.112,max_depth =3,n_estimator=50)

In [ ]:
xgb.fit(X,y)

C:\Users\dai\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [11:55:01] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimator" } are not used.

  warnings.warn(smsg, UserWarning)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.112, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimator=50, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [ ]:

params = {'learning_rate': np.linspace(0.001, 0.9, 10), 'max_depth': [None, 2, 3, 4], 'n_estimators': [25,50, 100]}
kfold = KFold(n_splits=5, shuffle=True, random_state=24)
xgb = XGBRegressor (random_state=24)
gcv = GridSearchCV(xgb, param_grid=params, cv=kfold, n_jobs=-1)
gcv.fit(X, y)

print(gcv.best_params_)
print(gcv.best_score_)

{'learning_rate': 0.10088888888888889, 'max_depth': 4, 'n_estimators': 50}
0.5961888902321231


In [264]:
train['Item_Weight']= train.groupby('Item_Identifier')['Item_Weight'].transform(lambda x: x.fillna(x.dropna().mean()))
print(train.sort_values(by='Item_Identifier',ascending=True).head())
test['Item_Weight']= test.groupby('Item_Identifier')['Item_Weight'].transform(lambda x: x.fillna(x.dropna().mean()))


KeyError: 'Column not found: Item_Weight'

In [265]:
print(train['Item_Weight'].isnull().sum())
print(train['Item_Weight'].notnull().sum())
print(train.head(20))

KeyError: 'Item_Weight'

In [266]:
train['Outlet_Size']=train['Outlet_Size'].fillna(method='ffill')

C:\Users\dai\AppData\Local\Temp\ipykernel_21924\1093344038.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train['Outlet_Size']=train['Outlet_Size'].fillna(method='ffill')


In [267]:
train['Item_Weight']=train['Item_Weight'].fillna(method='ffill')

KeyError: 'Item_Weight'

In [268]:
train.isnull().sum()

Item_Identifier              0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [175]:
test['Item_Fat_Content'].replace({'reg': 'Regular', 'LF': 'Low Fat','low fat' :'Low Fat'},inplace=True)
test['Item_Weight']=test['Item_Weight'].fillna(method='ffill')
test['Outlet_Size']=test['Outlet_Size'].fillna(method='ffill')

C:\Users\dai\AppData\Local\Temp\ipykernel_21924\2629763449.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test['Item_Weight']=test['Item_Weight'].fillna(method='ffill')
C:\Users\dai\AppData\Local\Temp\ipykernel_21924\2629763449.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test['Outlet_Size']=test['Outlet_Size'].fillna(method='ffill')


In [176]:
df = pd.concat([train, test])

In [177]:
df.isnull().sum()

Item_Identifier                  0
Item_Weight                      0
Item_Fat_Content                 0
Item_Visibility                  0
Item_Type                        0
Item_MRP                         0
Outlet_Identifier                0
Outlet_Establishment_Year        0
Outlet_Size                      0
Outlet_Location_Type             0
Outlet_Type                      0
Item_Outlet_Sales             5681
Item                         13709
dtype: int64

In [174]:
train['Item'] = train['Item_Weight'].drop_duplicates()
train['Item'].dropna()

0        9.300
1        5.920
2       17.500
3       19.200
4        8.930
         ...  
7953     6.115
8060     7.075
8320     8.395
8327    12.350
8345     7.470
Name: Item, Length: 495, dtype: float64